In [ ]:
import torch
import torchvision
import pandas as pd
import numpy as np

import json
import os
import unicodedata

import random
from transformers import set_seed

seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)

from tqdm import tqdm
import pymupdf
import pymupdf4llm
from collections import Counter

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

from accelerate import Accelerator

import langchain

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever
from langchain.document_loaders import PDFPlumberLoader, PyMuPDFLoader, PyPDFLoader, UnstructuredPDFLoader

import peft
from peft import PeftModel

import datasets
from datasets import Dataset
from transformers import Trainer, TrainingArguments

In [38]:
class Opt:
    def __init__(self):
        self.model_configs = {
            'skt/kogpt2-base-v2': {
                'quantization_config': BitsAndBytesConfig(
                    load_in_8bit=True,
                    bnb_4bit_compute_dtype=torch.bfloat16,
                ),
                'torch_dtype': torch.float16,
                'max_token': 512,
            },
        }

        self.llm_model = 'skt/kogpt2-base-v2'
        self.llm_model_config = self.model_configs[self.llm_model]
        self.llm_peft = False
        self.llm_peft_checkpoint = None

        self.embed_models = ['distilbert-base-uncased', 'google/mobilebert-uncased']
        self.embed_model = self.embed_models[1]

        self.pdf_loader = 'pymupdf'

        self.base_directory = './'
        self.train_csv_path = os.path.join(self.base_directory, 'train.csv')
        self.test_csv_path = os.path.join(self.base_directory, 'test.csv')
        self.chunk_size = 256
        self.chunk_overlap = 16

        self.ensemble = True
        self.bm25_w = 0.5
        self.faiss_w = 0.5

        self.is_submit = True
        self.eval_sum_mode = False

        self.output_dir = 'test_results'
        self.output_csv_file = f"{self.llm_model.replace('/', '_')}_{self.embed_model.split('/')[1]}_pdf{self.pdf_loader}_chks{self.chunk_size}_chkovp{self.chunk_overlap}_bm25{self.bm25_w}_faiss{self.faiss_w}_mix_submission.csv"
        os.makedirs(self.output_dir, exist_ok=True)

    def to_json(self):
        return json.dumps(self.__dict__)

args = Opt()

In [ ]:
def setup_llm_pipeline():
    tokenizer = AutoTokenizer.from_pretrained(args.llm_model)
    tokenizer.use_default_system_prompt = False

    model = AutoModelForCausalLM.from_pretrained(
        args.llm_model,
        quantization_config=args.llm_model_config['quantization_config'],
        torch_dtype=args.llm_model_config['torch_dtype'],
        device_map='auto',
        trust_remote_code=True
    )

    if args.llm_peft:
        model = PeftModel.from_pretrained(model, args.llm_peft_checkpoint)

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task='text-generation',
        return_full_text=False,
        max_new_tokens=args.llm_model_config['max_token']
    )

    return HuggingFacePipeline(pipeline=text_generation_pipeline)

In [9]:
def normalize_path(path):
    return unicodedata.normalize('NFC', path)

def format_docs(docs):
    return '\n'.join([doc.page_content for doc in docs])

def process_pdf(file_path):
    md_text = pymupdf4llm.to_markdown(file_path)
    header_split = [
        ('#', 'header I'),
        ('##', 'header II'),
        ('###', 'header III'),
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=header_split, strip_headers=False)
    md_chunks = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=args.chunk_size, chunk_overlap=args.chunk_overlap
    )

    splits = text_splitter.split_documents(md_chunks)
    return splits

In [ ]:
def create_vector_db(chunks, model_path, method='faiss'):
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def process_single_pdf(pdf_path):

    chunks = process_pdf(pdf_path)
    
    db = create_vector_db(chunks, model_path=args.embed_model)
    
    kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)
    faiss_retriever = db.as_retriever()

    retriever = EnsembleRetriever(
        retrievers=[kiwi_bm25_retriever, faiss_retriever],
        weights=[args.bm25_w, args.faiss_w],
        search_type='mmr'
    )

    del chunks, db, kiwi_bm25_retriever, faiss_retriever
    torch.cuda.empty_cache()

    return retriever

def process_questions_for_pdf(pdf_path, questions_df):

    retriever = process_single_pdf(pdf_path)

    llm_pipeline = setup_llm_pipeline()

    answers = []
    for _, row in questions_df.iterrows():
        question = row['Question']
        print(f"Generating answer for: {question}")

        result = llm_pipeline(question)
        # answers.append({
        #     'ID': row['SAMPLE_ID'],
        #     'Answer': result[0]['generated_text']
        # })

        if isinstance(result, list):
            answer_text = result[0]['generated_text'] if 'generated_text' in result[0] else result[0]
        elif isinstance(result, dict):
            answer_text = result.get('generated_text', result)
        else:
            answer_text = result 

        answers.append({
            'SAMPLE_ID': row['SAMPLE_ID'], 
            'Answer': answer_text
        })
    
    # 메모리 해제
    del retriever, llm_pipeline
    torch.cuda.empty_cache()

    return answers

In [13]:
def process_test_questions(df):

    all_answers = []
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc='Processing PDFs'):
        pdf_questions_df = df[df['Source_path'] == path]

        answers = process_questions_for_pdf(path, pdf_questions_df)
        all_answers.extend(answers)

    return all_answers

In [14]:
test = pd.read_csv('./test.csv')

In [15]:
test

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
...,...,...,...,...
93,TEST_093,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?
94,TEST_094,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
95,TEST_095,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게...
96,TEST_096,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,"우발부채란 무엇이며, 그 관리가 왜 중요한가?"


In [16]:
train = pd.read_csv("./train.csv")
train

,SAMPLE_ID,Source,Source_path,Question,Answer
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."
...,...,...,...,...,...
491,TRAIN_491,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,자치단체 보조금과 민간보조금은 각각 어떤 비율로 증가했는가?,"자치단체 보조금은 2019년 대비 2022년에 35% 증가하였고, 민간보조금은 10..."
492,TRAIN_492,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,한국의 재정금융안정계획이 주로 어떤 목표에 초점을 맞추어 있었는가?,한국의 재정금융안정계획은 통화량 조절과 물가안정이라는 단기적 목표에 초점을 맞추어 ...
493,TRAIN_493,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,1952년에 체결된 '한미경제조정협정'은 어떤 문제를 해결하기 위해 체결되었는가?,"원조물자 판매대금의 효과적 활용, 참전유엔군 경비지출을 위해 우리 정부에서 대여해 ..."
494,TRAIN_494,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,"2023년 12월 IMF는 성장 전망을 어떻게 제시하고 있으며, 성장세의 둔화가 어...","IMF는 최근 세계 경제전망을 통해 작년 3.5%에서 올해 3%, 내년 2.9%로 ..."


In [17]:
if __name__ == "__main__":
    test_df = pd.read_csv(args.test_csv_path)
    answers = process_test_questions(test_df)
    print(answers)

    # 결과 저장
    output_path = os.path.join(args.output_dir, args.output_csv_file)
    pd.DataFrame(answers).to_csv(output_path, index=False)

Processing PDFs:   0%|                                                                           | 0/9 [00:00<?, ?it/s]c:\users\dohyeong\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
No sentence-transformers model found with name google/mobilebert-uncased. Creating a new one with mean pooling.
c:\users\dohyeong\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\users\dohyeong

Generating answer for: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
Generating answer for: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
Generating answer for: 중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
Generating answer for: 2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
Generating answer for: 혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
Generating answer for: 재창업자금이 재도약지원자금으로 이관된 연도는 언제인가요?
Generating answer for: 창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요?
Generating answer for: 혁신창업사업화자금(융자) 사업을 시행하는 주체는 누구인가요?
Generating answer for: 혁신창업사업화자금(융자) 사업 집행절차는 어떻게 되나요?


Processing PDFs:  11%|███████▎                                                          | 1/9 [03:04<24:32, 184.10s/it]No sentence-transformers model found with name google/mobilebert-uncased. Creating a new one with mean pooling.
c:\users\dohyeong\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generating answer for: 부모급여 지원 사업의 목적은 무엇인가요?
Generating answer for: 부모급여(영아수당)의 2024년 확정된 예산은 몇백만원인가요?
Generating answer for: 부모급여 지원 사업은 어떤 법령상 근거를 갖고 추진되고 있나요?
Generating answer for: 영아수당 도입에 대한 추진경위는 어떻게 되나요?
Generating answer for: 부모급여 지원사업은 언제부터 시행되었나요?
Generating answer for: 보건복지부의 부모급여(영아수당) 지원 사업시행방법은 무엇이며, 사업 수혜자는 누구인가?


Processing PDFs:  22%|██████████████▋                                                   | 2/9 [04:53<16:19, 139.91s/it]No sentence-transformers model found with name google/mobilebert-uncased. Creating a new one with mean pooling.
c:\users\dohyeong\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Generating answer for: 재정성과관리제도는 어떤 측면에서 국정운영과 연결되는가?
Generating answer for: 성과관리의 실효성 강화를 위해 정부가 취한 조치는 무엇인가?
Generating answer for: 재정성과관리 관련 주요 쟁점은 무엇인가?
Generating answer for: 재정성과관리가 왜 중요한가?
Generating answer for: 재정성과관리는 무엇을 목표로 하는가?
Generating answer for: 어떤 국제기구들이 사업을 기준으로 예산을 나누어 성과 정보를 생산하고 있는가?
Generating answer for: 재정성과관리의 목적은 무엇인가?
Generating answer for: 2021년 「국가재정법」 개정으로 어떤 규정이 신설되었는가?
Generating answer for: 성과관리제도는 지출 구조조정을 위해 어떤 방향으로 추진되고 있는가?
Generating answer for: 재정사업 자율평가의 목적은 무엇이며, 어떤 방식으로 제도 개선이 이루어졌는가?
Generating answer for: 2016년 재정성과관리제도의 환류 개선사항은 무엇인가?
Generating answer for: 2018년도에 재정성과관리제도 개선사항과, 이로 인해 어떤 효과가 발생했는가?
Generating answer for: 재정사업 자율평가의 전면 개편을 통해 어떤 중점 추진과제가 제시되었는가?
Generating answer for: 재정성과관리제도의 중요성과 국정운영과의 연결성은 무엇인가?
Generating answer for: 재정성과관리체계 강화를 위해 정부가 어떤 제도를 제시했으며, 재정성과관리는 무엇을 목표로 하는가?
Generating answer for: 재정성과관리는 어떤 과정에서 수행되며, 무엇을 증진하기 위해 활동하는가?
Generating answer for: 재정성과관리의 목적과 우리나라의 재정성과관리제도가 프로그램 예산제도로 전환된 시기는?
Generating an

Processing PDFs:  89%|██████████████████████████████████████████████████████████▋       | 8/9 [24:08<03:39, 219.07s/it]No sentence-transformers model found with name google/mobilebert-uncased. Creating a new one with mean pooling.
c:\users\dohyeong\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generating answer for: 우리나라에서는 언제부터 발생주의 기준을 적용한 국가결산보고서에서 우발부채를 보고하고 있는가?
Generating answer for: 우발부채 관련 주요 쟁점은 무엇인가?
Generating answer for: 우발부채의 관리는 왜 중요한 이슈로 여겨지는가?
Generating answer for: 우발부채와 부채의 차이점은 무엇인가?
Generating answer for: 발생주의와 현금주의의 차이는 무엇인가?
Generating answer for: 채무지속가능성분석은 어떤 목적을 가지고 도입되었는가?
Generating answer for: 의제의무란 무엇인가?
Generating answer for: 국제통화기금이 재정통계 작성의 국제기준을 제시하기 위해 발간한 매뉴얼은 무엇인가?
Generating answer for: 계류중인 소송사건이란 무엇인가?
Generating answer for: 최소운영수입보장(BTO 등) 제도란 무엇을 의미하는가?
Generating answer for: 우발부채에 대한 내용으로 대표적으로 어떤 사항이 해당되는가?
Generating answer for: GFSM2014에서는 우발부채를 어떻게 구분하는가?
Generating answer for: GFSM은 몇 차례의 개정을 거쳤으며, 어떠한 목적으로 GFSM 2001이 개정되었는가?
Generating answer for: 표준화 보증이란 무엇인가?
Generating answer for: 표준화 보증에서 공공부문의 우발부채는 어떻게 인식되는가?
Generating answer for: 재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?
Generating answer for: 미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
Generating answer for: 국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게 다른가요?
Generating answer for: 우발부채

Processing PDFs: 100%|██████████████████████████████████████████████████████████████████| 9/9 [30:00<00:00, 200.03s/it]

[{'SAMPLE_ID': 'TEST_000', 'Answer': '?\n현재 370억원 수준입니다.\n이 자금은 기존 융자금의 상환기일까지 한시적으로 만료되어 만료됨에 따라 이자부담이 다소 높아질 것으로 보입니다.\n현재 시중은행이 저금리 대출상품을 활용하여 저금리 대출상품으로 대출을 하는 대출이 매우 적은 상황이기 때문에 융자금 상환일까지 이자부담이 거의 없는 상황에서 저금리 대출상품을 활용하여 저금리 대출상품을 활용해 저금리 대출상품으로 저금리 대출상품을 활용하여 대출 상환일까지 융자를 상환한다면 이자부담도 없어질 것입니다.\n현재 저금리 대출상품으로 저금리 전환을 선택할 시에는 금리부담이 거의 없습니다.\n금리가 다소 낮은 편이지만 대출 이자 부담이 만만하지 않다면 앞으로 금리 상승 시 금리 부담도 없을 것 입니다. 이번 조사는 대선 후보인 박근혜 전 대통령이 지난 9월 29일 제출한 ‘새누리당의 당협위원장 임명 동의 요청 건에 대한 답변’을 대상으로 한다.\n새누리당은 당초 2월4일부터 25일까지 ‘국민께 드리는 호소문’을 발송할 계획이었으나 박근혜 전 대통령과 공교롭게도 같은 날인 24일 중앙선관위와 중앙선거관리위원회에 접수된 ‘대통령 후보자의 공당별 추천에 대한 답변’ 문건은 발송하지 못하게 됐다.\n이런 결정은 박 전 대통령의 공당별 추천에 대한 답변을 앞두고 사전 선거운동 목적으로 이뤄지는 것이라 의미가 크다는 게 중앙선거관리위원회 판단이다.\n박 전 대통령은 선관위의 답변을 받은 뒤에도 중앙선관위에 선거운동 위반 등의 혐의로 출석 요구를 할 수 있는 상황이다.\n이에 선관위는 25일 해당 문건을 발송하지 않기로 결정했다.\n이는 선관위가 박 전 대통령과 대선 후보자의 답변을 검토한 뒤 서면 보고를 하게 한 것과 다르지 않다.\n중앙선관위는 “국민께 드리는 사안”이라는 제목의 답변에서 “박 전 대통령의 공당별 추천에 관한 공당별 추천은 사안에 따라 다를 수 있으며 공당별 추천에 대한 사전 사전 공표는 선관위 요청으로 이미 결정된 사항”이라며 “

In [18]:
test

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
...,...,...,...,...
93,TEST_093,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?
94,TEST_094,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
95,TEST_095,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게...
96,TEST_096,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,"우발부채란 무엇이며, 그 관리가 왜 중요한가?"


In [24]:
pd.read_csv("./test_results/skt_kogpt2-base-v2_mobilebert-uncased_pdfpymupdf_chks512_chkovp32_bm250.5_faiss0.5_mix_submission.csv")

,SAMPLE_ID,Answer
0,TEST_000,?\n현재 370억원 수준입니다.\n이 자금은 기존 융자금의 상환기일까지 한시적으로...
1,TEST_001,”라고 질문했다.\n창조경제 실현을 위한 청년창업가 육성정책을 수립·지원하겠다고 답...
2,TEST_002,???\n창업자금(융자) 사업실패 지원비용의 50~70%는 창업성공 때 이자부담을 ...
3,TEST_003,"""\n나는 청년창업사관학교 2기 창업동아리 수료생과 선배 벤처의 조언을 들어 보았다..."
4,TEST_004,`라고 물어보는 질문을 던졌다.\n이에 “미래산업과 관련해 (투자) 유망한 분야에 ...
...,...,...
93,TEST_093,""" 이 질문에 대답하면, 국가 전체의 신용위험을 보여주는 주요 원인에는 정부 보증에..."
94,TEST_094,"""\n""어느 정부 부처가 그 정부 부처의 이권 개입에 간섭하도록 되어 있다. 그런 ..."
95,TEST_095,"""\n""우발부채라는 말을 쓴지 10년도 안되서 다시 재정경제원에서도 우발채권을 얘기..."
96,TEST_096,여당이 이렇게까지 국민들을 외면하면서 어떤 정책적 노력을 펼치겠다고 했습니까?\n...


In [41]:
test.loc[1, 'Question']

'중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?'

In [42]:
pd.read_csv("./test_results/skt_kogpt2-base-v2_mobilebert-uncased_pdfpymupdf_chks512_chkovp32_bm250.5_faiss0.5_mix_submission.csv").loc[1, 'Answer']

'”라고 질문했다.\n창조경제 실현을 위한 청년창업가 육성정책을 수립·지원하겠다고 답변한 박 예비후보는 “우리도 창업지원프로그램이 잘 구축되고 있지만 정부의 창업진흥정책이 미흡한게 가장 큰 문제”라고 답했다. 이번 포럼은 ▲2013년 글로벌 금융위기 이후 한국의 경제정책 방향과 주요 이슈의 전망 ▲4차 산업혁명과 금융혁신을 위한 정책방향 ▲새 정부의 정책방향과 정책방향 등도 논의될 예정이다.\n또한 4차 산업혁명 시대에 글로벌 금융업계의 경쟁력을 향상시키고 이를 통해 새로운 비즈니스 모델을 창출하는 등 다양한 협력사업을 전개하기 위한 방안을 모색하는 토론의 시간도 마련된다.\n홍윤식 한국금융연구원 원장은 개회사를 통해 "4차 산업혁명과 핀테크혁신을 통해 미래 금융환경과 경제활력을 창출하기 위해 다양한 정책노력과 정책마련이 필요하다"라며 "3D 프린팅, 모바일뱅킹, P2P 등 다양한 산업군간의 경쟁력 강화와 신규 비즈니스 모델 창출을 위한 협력모델도 모색할 수 있는 계기가 돼야 한다"고 밝혔다.\n한편, 미래금융협회 회원들을 대상으로 한 이번 포럼 개최는 금융산업 발전과 함께 금융산업 발전을 위한 정책방향과 주요 정책방향, 디지털 경제혁신을 위한 주요 이슈 등을 논의하는 자리가 될 예정이다. 금융감독원은 25일 금융사기에 이용된 휴대폰 가입자와 대리점 등에 대한 현장조사 결과 이같이 확인됐다고 밝혔다.\n금감원은 지난 20일부터 전국 일선 증권사 412곳을 대상으로 현장조사를 벌여 이같은 사실을 파악하고 각 증권사에 안내했다.\n현장에선 가입자들을 대상으로 "휴대폰은 가입자 이름과 실명이 확인된 사실을 확인한 뒤 해당 증권사의 영업점이나 대리점에 전화하지 않는다"며 휴대폰 가입 여부와 비밀번호를 확인해 줄 것을 당부했다.\n또 "휴대폰이 아닌 휴대폰을 이용해 소액결제계좌를 개설한 고객은 비밀번호가 누설됐는지 확인할 필요가 있다"고 말했다.\n증권가 지점이나 대리점에 전화를 거는 등 불법개입한 경우는 모두 26만8000명(약 7만2000명)으로 집계됐고, 이 가운